In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Imposta il dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)  # Carica l'immagine
        label = self.data.iloc[idx, 1]  # 1 se "human", 0 se "nothuman"
        if self.transform:
            image = self.transform(image)
        return image, label

# Definisci le trasformazioni
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Flip orizzontale casuale
    transforms.RandomRotation(20),       # Rotazione casuale
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), # Modifiche ai colori
    transforms.RandomResizedCrop(224),
    transforms.ToTensor()
])

# Percorso del CSV e delle immagini
csv_file = '/content/folder/classi.csv'  # Sostituisci con il tuo file CSV
root_dir = '/content/folder'      # Sostituisci con il tuo percorso delle immagini

# Carica il dataset
dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

In [ ]:
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2)

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, sampler=train_sampler, batch_size=128)
val_loader = DataLoader(dataset, sampler=val_sampler, batch_size=128)

In [ ]:
model = models.resnet50()

# Modifica l'output del modello per la classificazione binaria
model.fc = nn.Linear(model.fc.in_features, 1)  # Un solo output per classificazione binaria

# Sposta il modello su GPU se disponibile
model = model.to(device)

# Definisci la loss function e l'ottimizzatore
criterion = nn.BCEWithLogitsLoss()  # Usa BCEWithLogitsLoss per problemi binari
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Funzione per allenare il modello
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()  # Assicurati che le etichette siano float

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()  # Rimuovi la dimensione extra
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Stampa la loss media per l'epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

        # Valuta il modello
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float()
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                predicted = torch.round(torch.sigmoid(outputs))  # Applica sigmoid e arrotonda per ottenere le predizioni binarie
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%')

# Avvia l'allenamento
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

Epoch 1/50, Loss: 0.7881908416748047
Validation Loss: 3.9879138469696045, Accuracy: 57.142857142857146%
Epoch 2/50, Loss: 1.652338981628418
Validation Loss: 0.6948969960212708, Accuracy: 57.142857142857146%
Epoch 3/50, Loss: 0.9131397008895874
Validation Loss: 0.697255551815033, Accuracy: 57.142857142857146%
Epoch 4/50, Loss: 0.5015282034873962
Validation Loss: 0.7340145707130432, Accuracy: 57.142857142857146%
Epoch 5/50, Loss: 0.9616330862045288
Validation Loss: 0.8159098029136658, Accuracy: 28.571428571428573%
Epoch 6/50, Loss: 0.784874677658081
Validation Loss: 0.8670950531959534, Accuracy: 42.857142857142854%
Epoch 7/50, Loss: 1.0382381677627563
Validation Loss: 1.087705135345459, Accuracy: 42.857142857142854%
Epoch 8/50, Loss: 0.5176028609275818
Validation Loss: 1.4306769371032715, Accuracy: 42.857142857142854%
Epoch 9/50, Loss: 0.3923184275627136
Validation Loss: 1.9493615627288818, Accuracy: 42.857142857142854%
Epoch 10/50, Loss: 0.47633096575737
Validation Loss: 2.2212598323822

In [ ]:
# Salva l'intero modello
torch.save(model, 'full_model.pth')

In [ ]:
# Salva i pesi del modello
torch.save(model.state_dict(), 'model_weights.pth')